In [ ]:
import numpy as np
from scipy.optimize import fsolve
import re
from numpyarray_to_latex import to_ltx
from IPython.display import display, Math

%load_ext autoreload

def disp(*mats, prefix=""):
    total_str = prefix
    for mat in mats:
        mat_obj = globals()[mat]
        ltx_str = to_ltx(mat_obj, fmt="{:.0f}")
        total_str = fr"{total_str}\ {mat}\ =\ {ltx_str}"

    display(Math(total_str))


def make_parity_and_order(n, k):
    order = [0] * n

    data_idx = 0
    parity_exp = 0
    parity_idx = k

    for i in range(n):
        if i+1 == 2 ** parity_exp:
            order[i] = parity_idx
            parity_idx += 1
            parity_exp += 1
        else:
            order[i] = data_idx
            data_idx += 1
    
    # Every number up to [1, n] that is not a power of 2
    P = np.arange(1, n+1, dtype=np.uint8)
    P = P[np.log2(P) != np.floor(np.log2(P))]
    P = np.unpackbits(P.reshape((k, 1)), axis=1, count=r, bitorder="little")

    return P, order

$$
\begin{split}
2^r &\geq k+r+1  \\
f(x) &= 2^x-x-k-1 \\
r &= \lceil{x}\rceil \mid f(x) = 0,\ x > 0 \\
\end{split}
$$

In [92]:
# Information bits
k = 4

def parity_bits_fn(x):
    return (2 ** x) - x - k - 1

# Parity bits
x = fsolve(parity_bits_fn, x0=k).round(6).item()
r = int(np.ceil(x))

# Codeword length
n = k + r

display(Math(rf"k={k}\\r={r}\\n={n}"))

<IPython.core.display.Math object>

In [93]:
# Identity matrices
I_k = np.eye(k)
I_n_k = np.eye(n-k)

# Parity equations
P, order = make_parity_and_order(n, k)

# Generator matrix (k * n)
G = np.hstack((I_k, P))

# Parity-check matrix (k-k * n)
H = np.hstack((P.T, I_n_k))

G = G[:, order]
H = H[:, order]

G_t = G.T
disp("G_t", "H")

<IPython.core.display.Math object>

In [94]:
# Input vector
_ = np.array([np.random.randint(0, 1 << k)], dtype=np.uint8)
w = np.unpackbits(_, count=k, bitorder="little").reshape(k, 1)

# Codeword
c = G_t @ w % 2

disp("w", "c")

<IPython.core.display.Math object>

In [95]:

loc = np.random.randint(0, n) + 1
r = np.copy(c).astype("u8")
r[loc-1] ^= 1

# Syndrome vector
z = H @ r % 2
disp("r", "z")

print(f"Error in Bit #{loc}")
detected_loc = np.packbits(z.astype("u8"), bitorder='little').item()
print(f"Syndrome detected error in Bit #{detected_loc}")

<IPython.core.display.Math object>

Error in Bit #4
Syndrome detected error in Bit #4
